# PRÁCTICA SPARK: INFLUENCIA DEL COVID-19 EN BICIMAD
## Arturo Precioso Garcelán, Lucía Navarro Martín y Clara Queipo Álvarez

El objetivo de la presente práctica consiste en identificar las consecuencias del COVID-19 en el servicio de BICIMAD. Para ello analizaremos los datos del mes de junio de los años 2019 (periodo previo al COVID-19), 2020 (recién finalzado el confinamiento) y 2021 (año que todavía sufre los daños económicos sociales y eocnómicos de la pandemia pero que poco a poco se va imponiendo la "normalidad"). También analizaremos los datos del mes de diciembre antes y después del COVID-19. Para ello observaremos el mes de diciembre de los años 2018 (el conjunto de datos de 2019 tiene variables ligeramente diferentes y el análisis no podrá ser tan exhaustivo) y 2020. 

Los conjuntos de datos han sido extraídos de la página web del Ayuntamiento de Madrid https://opendata.emtmadrid.es/Datos-estaticos/Datos-generales-(1). Hemos utilizado los conjuntos de datos completos los cuales están subidos a cqueipo@picluster02.mat.ucm.es junto a este archivo y al programa .py.

Los datos de la base de datos presentan las siguientes variables:

- **id_**: identificador del movimiento
- **ageRange**: Número que indica el rango de edad del usuario. Sus posibles valores son:
        - 0: No se ha podido determinar el rango de edad del usuario
        - 1: El usuario tiene entre 0 y 16 años
        - 2: El usuario tiene entre 17 y 18 años
        - 3: El usuario tiene entre 19 y 26 años
        - 4: El usuario tiene entre 27 y 40 años
        - 5: El usuario tiene entre 41 y 65 años
        - 6: El usuario tiene 66 años o más
- **idplug_base**: Número de la base en la que se engancha la bicicleta.
- **idplug_station**: Número de la estación en la que se engancha la bicicleta.
- **idunplug_base**: Número de la base de la que se desengancha la bicicleta.
- **idunplug_station**: Número de la estación de la que se desengancha la bicicleta.
- **travel_time**: Tiempo total en segundos, entre el desenganche y el enganche de la bicicleta.
- **unplug_hourTime**: Franja horaria en la que se realiza el desenganche de la bicicleta. (Por cuestiones de anonimato, se facilita la hora de inicio del movimiento, sin la información de minutos y segundos. Todos los movimientos iniciados durante la misma hora, tendrán el mismo dato de inicio.)
- **user_day_code**: Código del usuario. (Para una misma fecha, todos los movimientos de un mismo usuario, tendrán el mismo código, con el fin de poder realizar estudios estadísticos de las tendencias diarias de los usuarios.)
- **user_type**: Número que indica el tipo de usuario que ha realizado el movimiento. Sus posibles valores son:
        - 0: No se ha podido determinar el tipo de usuario
        - 1: Usuario anual (poseedor de un pase anual)
        - 2: Usuario ocasional (perfil turista)
        - 3: Trabajador de la empresa
        - 6: Usuario Ocasional (no Abonado)
        - 7: Usuario de BiciMAD GO.
- **zip_code**: Texto que indica el código postal del usuario que ha realizado el movimiento.

In [96]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
import json
from pprint import pprint

En primer lugar vamos a analizar el uso del servicio BICIMAD en el mes de junio de los años 2019 y 2020. 
Nuestro objetivo es ver los impactos del COVID-19 en el servicio de BICIMAD. Posteriormente analizaremos las cifras del año 2021.

In [97]:
junio2019 = spark.read.json('201906.json')
print ('- JUNIO 2019')
junio2019.show()
junio2019.count()

- JUNIO 2019
+---------------+--------------------+--------+-----------+--------------+-------------+----------------+-----+-----------+--------------------+--------------------+---------+--------+
|_corrupt_record|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|track|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+---------------+--------------------+--------+-----------+--------------+-------------+----------------+-----+-----------+--------------------+--------------------+---------+--------+
|           null|{5cf83b752f3843a0...|       0|         21|            66|            8|              90| null|        219|{2019-06-01T00:00...|e4d55deb9ac172a8d...|        1|        |
|           null|{5cf83b762f3843a0...|       4|         19|           136|           19|              71| null|        359|{2019-06-01T00:00...|8a0c4123e924a50a9...|        1|   28039|
|           null|{5cf83b762f3843a0...|       4|         17|   

450811

In [166]:
junio2020 = spark.read.json('202006.json')
print ('- JUNIO 2020')
junio2020.show()
junio2020.count()

- JUNIO 2020
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|{5eec97ff56e7da53...|       0|          4|            91|            2|             131|        674|2020-06-01T00:00:00Z|b1d54ac58fd5e41ac...|        1|        |
|{5eec97ff56e7da53...|       0|          9|           158|            1|               5|        426|2020-06-01T00:00:00Z|da5166ba006c7a78e...|        1|        |
|{5eec97ff56e7da53...|       0|         18|           129|            8|               5|        931|2020-06-01T00:00:00Z|9fcc3cb6297cd2f9a...|        1|        |
|{5eec97f

210555

In [99]:
print ('Número de viajes en 2019: ' + str(junio2019.count()))
print ('Número de viajes en 2020: ' + str(junio2020.count()))

Número de viajes en 2019: 450811
Número de viajes en 2020: 210555


Observamos que el número de viajes en 2019 es más del doble de 2020. Esto puede deberse a varios factores que observaremos más adelante, entre ellos
las restricciones en relación a los turistas que existía en junio de 2020 así como el predominio del teletrabajo y el temor a salir a la calle por miedo a contagiarse.
A continuación vamos a observar los distintos grupos en los que se dividen a los usuarios.

In [100]:
print ('- JUNIO 2019:')
junio2019.groupBy('user_type').count().show()
print ('- JUNIO 2020:')
junio2020.groupBy('user_type').count().show()

- JUNIO 2019:
+---------+------+
|user_type| count|
+---------+------+
|     null| 22865|
|        1|397252|
|        3| 25595|
|        2|  5099|
+---------+------+

- JUNIO 2020:
+---------+------+
|user_type| count|
+---------+------+
|        1|198186|
|        3| 10445|
|        2|  1924|
+---------+------+



A la vista de los resultados obtenidos afirmamos que el tipo de usuario más frecuente en el servicio BICIMAD es el número 1, es decir, el abonado.
Observamos que el número de usuarios anuales es bastante mayor en junio de 2019. El grupo "null" que aparece en 2019 es indeterminado, es decir, la recogida de datos ha podido sufrir algún tipo de problema y desconocemos si son abonados, turistas o trabajadores de empresa. 
El número de usuarios "turista" es bastante superior en el año 2019. Esto es lógico puesto que en 2020, como se ha expresado antes, no se podía viajar entre países y el turismo tan solo era nacional (algunas comunidades autónomas también restringieron moviminetos). En 2019 el número de usuarios "trabajador de empresa" también superó al de 2020 puesto que se impuso el teletrabajo y muchos empleados tuvieron que ir al ERTE.

In [101]:
print ('- JUNIO 2019:')
junio2019.groupBy('ageRange').count().show()
print ('- JUNIO 2020:')
junio2020.groupBy('ageRange').count().show()

- JUNIO 2019:
+--------+------+
|ageRange| count|
+--------+------+
|       0|189919|
|    null| 22865|
|       6|  2274|
|       5| 89417|
|       1|  3501|
|       3| 22642|
|       2|  1822|
|       4|118371|
+--------+------+

- JUNIO 2020:
+--------+------+
|ageRange| count|
+--------+------+
|       0|121486|
|       6|  1287|
|       5| 34958|
|       1|  1750|
|       3|  8328|
|       2|   893|
|       4| 41853|
+--------+------+



In [ ]:
Vemos que en ambos años lo que predomina es el grupo 0 (no se ha podido determinar el grupo de edad) seguido del grupo 4 (entre 27 y 40 años).

In [ ]:
Ahora vamos a analizar el número de movimientos en las estaciones

In [102]:
idplug2019 = junio2019.groupBy('idplug_station').count()
idplug2020 = junio2020.groupBy('idplug_station').count()

In [106]:
print ('- JUNIO 2019:')
idplug2019.filter(idplug2019['count']>5000).show()
print ('- JUNIO 2021:')
idplug2020.filter(idplug2020['count']>2500).show()

- JUNIO 2019:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           149| 5483|
|            57| 5323|
|          null|22865|
|           175| 5773|
|           163| 6495|
|           135| 5975|
|           129| 5993|
|            90| 5148|
+--------------+-----+

- JUNIO 2021:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            43| 2859|
|           132| 2586|
|           175| 3067|
+--------------+-----+



Las estaciones en las que más se engachan bicicletas en **JUNIO de 2019** son:
- 175: Calle Jaime el Conquistador, 30
- 163: Paseo de la Esperanza, 83
- 135: Plaza de San Juan de la Cruz

Algunas de las estaciones en las que más se engachan bicicletas en **JUNIO de 2020** son:
- 43: Calle de Lavapiés, 36
- 132: Calle de Santa Engracia, 168
- 175: Calle Jaime el Conquistador, 30

Vemos que tan solo la estación 175 coincide en ambos años.

In [107]:
idunplug2019 = junio2019.groupBy('idunplug_station').count()
idunplug2020 = junio2020.groupBy('idunplug_station').count()

In [111]:
print ('- JUNIO 2019:')
idunplug2019.filter(idunplug2019['count']>5500).show()
print ('- JUNIO 2021:')
idunplug2020.filter(idunplug2020['count']>2500).show()

- JUNIO 2019:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|            null|22865|
|             175| 5708|
|             163| 6454|
|             135| 5790|
|             129| 5921|
+----------------+-----+

- JUNIO 2021:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              43| 2824|
|             132| 2576|
|             175| 3102|
+----------------+-----+



Las estaciones en las que más se desengachan bicicletas en **JUNIO de 2019** son:
- 175: Calle Jaime el Conquistador, 30
- 163: Paseo de la Esperanza, 83
- 135: Plaza de San Juan de la Cruz

Algunas de las estaciones en las que más se desengachan bicicletas en **JUNIO de 2020** son:
- 43: Calle de Lavapiés, 36
- 132: Calle de Santa Engracia, 168
- 175: Calle Jaime el Conquistador, 30

Las estaciones que más se desenganchan coinciden con las que más se enganchan. Vemos que tan solo la estación 175 coincide en ambos años. 

Afirmamos que las estaciones más utilizadas en junio de 2019 son 175 y 163 y en junio de 2020 son 43 y 175.

In [115]:
print ('- JUNIO 2019:')
idplug2019.filter(idplug2019['count']<500).show()
print ('- JUNIO 2021:')
idplug2020.filter(idplug2020['count']<330).show()

- JUNIO 2019:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            28|  434|
|          2008|    6|
|           207|  339|
|           183|  435|
+--------------+-----+

- JUNIO 2021:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            28|  214|
|           210|  304|
|           151|  300|
+--------------+-----+



Las estaciones en las que menos se enganchan en **JUNIO de 2019** son:
- 207: Calle de las Azucenas,1  
- 28: Calle de Alcalá, 26
- 183: Calle de Hermosilla, 135

Las estaciones en las que menos se enganchan en **JUNIO de 2020** son:
- 210: Calle de las Tres Cruces, 135
- 28: Calle de Alcalá, 26
- 151: Calle de Orense, 10

La estación 2008 no se encuentra en el archivo excel, ya que sólo tenemos 261 estaciones. A causa de esto, no podemos dar una ubicación exacta de dicha estación.
Puede ser un error de recogida o transcripción de datos. Vemos que solo coincide la estación 28.

In [120]:
print ('- JUNIO 2019:')
idunplug2019.filter(idunplug2019['count']<500).show()
print ('- JUNIO 2020:')
idunplug2020.filter(idunplug2020['count']<330).show()

- JUNIO 2019:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              28|  478|
|            2008|    6|
|             207|  296|
|             183|  413|
+----------------+-----+

- JUNIO 2020:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              28|  220|
|             210|  323|
|             151|  299|
+----------------+-----+



Las estaciones en las que menos se desenganchan en **JUNIO de 2019** son:
- 207: Calle de las Azucenas,1  
- 28: Calle de Alcalá, 26
- 183: Calle de Hermosilla, 135

Las estaciones en las que menos se desenganchan en **JUNIO de 2020** son:
- 210: Calle de las Tres Cruces, 135
- 28: Calle de Alcalá, 26
- 151: Calle de Orense, 10

Las estaciones que menos se desenganchan coinciden con las que más se enganchan. Vemos que tan solo la estación 28 coincide en ambos años. 

Afirmamos que las estaciones menos utilizadas en junio de 2019 son 28 y 207 y en junio de 2020 son 28 y 151.

In [ ]:
Tiempo de uso según el tipo de  usuario:

In [121]:
print ('- JUNIO 2019:')
junio2019.groupBy('user_type').sum('travel_time').show()
print ('- JUNIO 2O20:')
junio2020.groupBy('user_type').sum('travel_time').show()

- JUNIO 2019:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|     null|            null|
|        1|       349718610|
|        3|        71729157|
|        2|        11663511|
+---------+----------------+

- JUNIO 2O20:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        1|       229144789|
|        3|        41602542|
|        2|         5181881|
+---------+----------------+



El tipo de usuario que más tiempo utiliza el empleo es el 1. Después el 3 y finalmente el turista. Esto es consecuente con el número de trayectos de cada tipo de usuario.

AHORA VAMOS A ANALIZAR EL MES DE JUNIO DE AÑO 2021 PARA OBSERVAR SI HUBO UN CRECIMIENTO DEL SERVICIO BICIMAD TRAS JUNIO DE 2020

In [143]:
junio2021 = spark.read.json('202106.json')
print ('- JUNIO 2021')
junio2021.show()
junio2021.count()

- JUNIO 2021
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|{60bbf3f21b91311d...|       0|         15|            54|           13|              38|        319|2021-06-01T00:00:00Z|b67ea824acf706087...|        1|        |
|{60bbf3f21b91311d...|       0|          7|           126|            1|             185|        386|2021-06-01T00:00:00Z|364b1b99f1f9a1748...|        1|        |
|{60bbf3f21b91311d...|       0|         14|            91|           10|              15|        465|2021-06-01T00:00:00Z|5cc17031cba02835d...|        1|        |
|{60bbf3f

413370

In [123]:
print ('Número de viajes en 2021: ' + str(junio2021.count()))

Número de viajes en 2021: 413370


Vemos que el número de viajes es bastante más elevado que en 2020 pero no alcanza las cifras de 2019.

In [124]:
print ('- JUNIO 2021:')
junio2021.groupBy('user_type').count().show()

- JUNIO 2021:
+---------+------+
|user_type| count|
+---------+------+
|        6|  4847|
|        1|391401|
|        3| 15168|
|        2|  1951|
|        7|     3|
+---------+------+



El tipo de usuario más frecuente de nuevo es el abonado. Vemos que aparecen nuevos tipos de usuarios: el grupo 6 y 7 que no estaban en 2019 ni en 2020. 

In [144]:
print ('- JUNIO 2021:')
junio2021.groupBy('ageRange').count().show()

- JUNIO 2021:
+--------+------+
|ageRange| count|
+--------+------+
|       0|229807|
|       6|  2052|
|       5| 76926|
|       1|  4789|
|       3| 13208|
|       2|  1138|
|       4| 85450|
+--------+------+



Vemos que de nuevo el grupo más frecuente es el 0 seguido del grupo 4.

In [126]:
idplug2021 = junio2021.groupBy('idplug_station').count()

In [130]:
print ('- JUNIO 2021:')
idplug2021.filter(idplug2021['count']>3700).show()

- JUNIO 2021:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            57| 4503|
|            43| 4868|
|           175| 3758|
+--------------+-----+



Las estaciones que más se engachan bicicletas en **JUNIO de 2021** son:
- 43: Calle de Lavapiés, 36
- 57: Calle de Santa Cruz de Marcenado, 20
- 175: Calle Jaime el Conquistador, 30

In [165]:
idunplug2021 = junio2021.groupBy('idunplug_station').count()
print ('- JUNIO 2021:')
idunplug2021.filter(idunplug2021['count']>3700).show()

- JUNIO 2021:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|              57| 4516|
|              43| 4816|
|             175| 3719|
+----------------+-----+



Las estaciones en las que más se desengachan bicicletas en **JUNIO de 2021** son:
- 43: Calle de Lavapiés, 36
- 57: Calle de Santa Cruz de Marcenado, 20
- 175: Calle Jaime el Conquistador, 30

La estación más utilizada en junio de 2021 es la 43.

In [131]:
idunplug2021 = junio2021.groupBy('idunplug_station').count()

In [136]:
print ('- JUNIO 2021:')
idplug2021.filter(idplug2021['count']<400).show()

- JUNIO 2021:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           266|  225|
|           257|  266|
|            28|  301|
|          2009|   18|
|           225|  373|
+--------------+-----+



Las estaciones en las que menos se engachan bicicletas en **JUNIO de 2021** son:
- 28: Calle de Alcalá, 26
- 225: Calle de la Sierra Toledana, 33
- 257: Avenida de Juan de Herrera, 1

Las estaciones 266 y 2009 no se encuentran en el archivo excel, ya que sólo tenemos 261 estaciones. A causa de esto, no podemos dar una ubicación exacta de dicha estación.
Puede ser un error de recogida o transcripción de datos.

In [138]:
print ('- JUNIO 2021:')
idunplug2021.filter(idunplug2021['count']<400).show()

- JUNIO 2021:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             266|  222|
|              28|  311|
|             257|  333|
|             225|  394|
|            2009|   22|
+----------------+-----+



Las estaciones en las que menos se desengachan bicicletas en **JUNIO de 2021** son:
- 28: Calle de Alcalá, 26
- 225: Calle de la Sierra Toledana, 33
- 257: Avenida de Juan de Herrera, 1

Las estaciones 266 y 2009 no se encuentran en el archivo excel, ya que sólo tenemos 261 estaciones. A causa de esto, no podemos dar una ubicación exacta de dicha estación.
Puede ser un error de recogida o transcripción de datos.

Las estaciones menos utilizadas en junio de 2021 son la 28 y la 257.

In [147]:
print ('- JUNIO 2021:')
junio2021.groupBy('user_type').sum('travel_time').show()

- JUNIO 2021:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        6|         8913375|
|        1|       377692250|
|        3|        35204394|
|        2|         4561006|
|        7|            3673|
+---------+----------------+



De nuevo el usuario que más tiempo de viaje tiene es el tipo 1, el abonado.

AHORA ANALIZAMOS EL MES DE DICIEMBRE DE 2018 Y 2020 (2021 AUN NO HAY DATOS y 2019 el conjunto de datos es diferente)

In [148]:
dic2018 = spark.read.json('201812.json')
print ('- DICIEMBRE 2018')
dic2018.show()
dic2018.count()

- DICIEMBRE 2018
+---------------+--------------------+--------+-----------+--------------+-------------+----------------+--------------------+-----------+--------------------+--------------------+---------+--------+
|_corrupt_record|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|               track|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+---------------+--------------------+--------+-----------+--------------+-------------+----------------+--------------------+-----------+--------------------+--------------------+---------+--------+
|           null|{5c085aae2f384324...|       5|          7|            75|            7|              75|                null|         16|{2018-12-01T00:00...|39d847f3af0e9d8ab...|        3|        |
|           null|{5c085aae2f384324...|       4|         19|           128|           14|              57|{[{{[-3.6949683, ...|        262|{2018-12-01T00:00...|697479f320b109464...|   

273272

In [65]:
dic2020 = spark.read.json('202012.json')
print ('- DICIEMBRE 2020')
dic2020.show()
dic2020.count()

- DICIEMBRE 2020
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|                 _id|ageRange|idplug_base|idplug_station|idunplug_base|idunplug_station|travel_time|     unplug_hourTime|       user_day_code|user_type|zip_code|
+--------------------+--------+-----------+--------------+-------------+----------------+-----------+--------------------+--------------------+---------+--------+
|{5fcc10fc1b91315b...|       4|         16|           182|            8|              71|        337|2020-12-01T00:00:00Z|1b9453cb0a60d9826...|        1|   28012|
|{5fcc10fc1b91315b...|       0|         19|            18|            1|              35|        190|2020-12-01T00:00:00Z|29fe8cf0fb57fdfc4...|        1|        |
|{5fcc10fc1b91315b...|       4|         14|           175|           21|              46|        354|2020-12-01T00:00:00Z|051f14e387dd8f544...|        1|   28012|
|{5fc

265261

In [66]:
print ('Número de viajes en 2018: ' + str(dic2018.count()))
print ('Número de viajes en 2020: ' + str(dic2020.count()))

Número de viajes en 2018: 67166
Número de viajes en 2020: 265261


A la vista de los resultados obtenidos observamos que el número de viajes en diciembre de 2018 es muy inferior a los viajes de 2020, lo cual nos llama bastante la atención puesto que anteriormente vimos que en junio de 2020 el número de viajes era bastante inferior a los de 2019. Esto puede deberse a que tras el confinamiento, muchos usuarios decidieron prescindir del transporte público debido al acumulamineto de gente y el temor a contagiarse. 

Vemos también que el número de usuarios en diciembre de 2020 es superior al número de usuarios en junio de 2020. Teniendo en cuenta que el servicio BICIMAD en los meses más fríos del año es menos utilizado es llamativo que en el año 2020 sea al revés. Esto es debido también al COVID-19. En el mes de junio muchos trabajadores todavía trabajaban desde casa y los turistas internacionales no podían visitar la ciudad. En cambio, en el mes de diciembre, al mejorar las condiciones sanitarias, algunos trabajadores se reincorporaron a sus puestos de trabajo y eligieron el transporte con BICIMAD al llevarse a cabo al aire libre.

In [149]:
print ('- DICIEMBRE 2018:')
dic2018.groupBy('user_type').count().show()
print ('- DICIEMBRE 2020:')
dic2020.groupBy('user_type').count().show()

- DICIEMBRE 2018:


+---------+------+
|user_type| count|
+---------+------+
|     null| 43605|
|        1|208055|
|        3| 18939|
|        2|  2673|
+---------+------+

- DICIEMBRE 2020:
+---------+------+
|user_type| count|
+---------+------+
|        6|  1345|
|        1|255154|
|        3|  8314|
|        2|   446|
|        7|     2|
+---------+------+



El usuario más frecuente, como en casos anteriores es el abonado. En cuanto a los turistas, podemos observar que, a pesar de que en 2020 hubo más trayectos, estos se llevaron a cabo por menos turistas que en diciembre de 2018 por las restricciones sanitarias nombradas anteriormente.

In [68]:
print ('- DICIEMBRE 2018:')
dic2018.groupBy('ageRange').count().show()
print ('- DICIEMBRE 2020:')
dic2020.groupBy('ageRange').count().show()

- DICIEMBRE 2018:
+--------+-----+
|ageRange|count|
+--------+-----+
|       0|24172|
|    null| 9749|
|       6|  230|
|       5|12045|
|       1|  762|
|       3| 3135|
|       2|  590|
|       4|16483|
+--------+-----+

- DICIEMBRE 2020:
+--------+------+
|ageRange| count|
+--------+------+
|       0|144033|
|       6|  1310|
|       5| 49799|
|       1|  2221|
|       3| 10573|
|       2|  1046|
|       4| 56279|
+--------+------+



El grupo de edad más predominante es nuevamente el grupo 4.

In [73]:
idplug2018 = dic2018.groupBy('idplug_station').count()
idplug_2020 = dic2020.groupBy('idplug_station').count()

In [154]:
print ('- DICIEMBRE 2018:')
idplug2018.filter(idplug2018['count']>880).show()
print ('- DICIEMBRE 2020:')
idplug_2020.filter(idplug_2020['count']>2800).show()

- DICIEMBRE 2018:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|          null| 9749|
|           175|  944|
|           163|  889|
|           135|  882|
+--------------+-----+

- DICIEMBRE 2020:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            57| 3251|
|            43| 3260|
|            46| 2890|
+--------------+-----+



Las estaciones en las que más se engachan bicicletas en **DICIEMBRE de 2018** son:
- 175: Calle Jaime el Conquistador, 30
- 163: Paseo de la Esperanza, 83
- 135: Plaza de San Juan de la Cruz

Las estaciones en las que más se engachan bicicletas en **DICIEMBRE de 2020** son:
- 43: Calle de Lavapiés, 36
- 46: Calle de la Ribera de Curtidores, 28
- 57: Calle de Santa Cruz de Marcenado, 20


In [157]:
idunplug2018 = dic2018.groupBy('idplug_station').count()
idunplug_2020 = dic2020.groupBy('idplug_station').count()

In [164]:
print ('- DICIEMBRE 2018:')
idunplug2018.filter(idunplug2018['count']>3300).show()
print ('- DICIEMBRE 2020:')
idunplug_2020.filter(idunplug_2020['count']>2800).show()

- DICIEMBRE 2018:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            43| 3488|
|          null|43605|
|           175| 3408|
|           135| 3425|
|           129| 3457|
+--------------+-----+

- DICIEMBRE 2020:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|            57| 3251|
|            43| 3260|
|            46| 2890|
+--------------+-----+



Las estaciones en las que más se desengachan bicicletas en **DICIEMBRE de 2018** son:
- 175: Calle Jaime el Conquistador, 30
- 43: Calle de Lavapiés, 36
- 135: Plaza de San Juan de la Cruz

Las estaciones en las que más se desengachan bicicletas en **DICIEMBRE de 2020** son:
- 43: Calle de Lavapiés, 36
- 46: Calle de la Ribera de Curtidores, 28
- 57: Calle de Santa Cruz de Marcenado, 20

Las estaciones más utilizadas en diciembre de 2018 son 175 y 135 y en diciembre de 2020 son 43 y 57.

In [158]:
print ('- DICIEMBRE 2018:')
idplug2018.filter(idplug2018['count']<100).show()
print ('- DICIEMBRE 2020:')
idplug_2020.filter(idplug_2020['count']<12).show()

- DICIEMBRE 2018:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|           119|   99|
|            29|    4|
|            28|    1|
+--------------+-----+

- DICIEMBRE 2020:
+--------------+-----+
|idplug_station|count|
+--------------+-----+
|          2008|    2|
|          3000|    1|
|           262|   11|
|           266|    2|
|           257|    6|
|           249|    5|
|           265|    2|
|           256|   11|
|           263|    9|
+--------------+-----+



Las estaciones en las que menos se engachan bicicletas en **DICIEMBRE de 2018** son:
- 28: Calle de Alcalá, 26
- 119: Calle de Méndez Álvaro, 7
- 29: Calle del Marqués de Cubas, 25

Las estaciones en las que menos se engachan bicicletas en **DICIEMBRE de 2020** son:
- 257: Avenida de Juan de Herrera, 1
- 249: Paseo de la Castellana, 298
- 256: Calle de Beatriz de Bobadilla, 2

Las estaciones 266, 262, 265, 263, 3000 y 2008 no se encuentran en el archivo excel, ya que sólo tenemos 261 estaciones. A causa de esto, no podemos dar una ubicación exacta de dicha estación. Puede ser un error de recogida o transcripción de datos.

In [93]:
print ('- DICIEMBRE 2018:')
idunplug2018.filter(idunplug2018['count']<110).show()
print ('- DICIEMBRE 2020:')
idunplug_2020.filter(idunplug_2020['count']<15).show()

- DICIEMBRE 2018:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|             119|  101|
|             120|  109|
|              29|    4|
|              28|    1|
+----------------+-----+

- DICIEMBRE 2020:
+----------------+-----+
|idunplug_station|count|
+----------------+-----+
|            2008|    3|
|             266|    5|
|             257|    5|
|             249|   12|
|             265|    3|
|             256|   13|
|             263|    7|
+----------------+-----+



Las estaciones en las que más se desengachan bicicletas en **DICIEMBRE de 2018** son:
- 28: Calle de Alcalá, 26
- 119: Calle de Méndez Álvaro, 7
- 29: Calle del Marqués de Cubas, 25

Las estaciones en las que más se desengachan bicicletas en **DICIEMBRE de 2020** son:
- 257: Avenida de Juan de Herrera, 1
- 249: Paseo de la Castellana, 298
- 256: Calle de Beatriz de Bobadilla, 2

Las estaciones 266, 265, 263 y 2008 no se encuentran en el archivo excel, ya que sólo tenemos 261 estaciones. A causa de esto, no podemos dar una ubicación exacta de dicha estación. Puede ser un error de recogida o transcripción de datos.

Las estaciones menos utilizadas en diciembre de 2018 son 28 y 29 y en diciembre de 2020 257 y 249.

In [86]:
print ('- DICIEMBRE 2018:')
dic2018.groupBy('user_type').sum('travel_time').show()
print ('- DICIEMBRE 2020:')
dic2020.groupBy('user_type').sum('travel_time').show()

- DICIEMBRE 2018:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|     null|            null|
|        1|        44432571|
|        3|        15689694|
|        2|          977117|
+---------+----------------+

- DICIEMBRE 2020:
+---------+----------------+
|user_type|sum(travel_time)|
+---------+----------------+
|        6|         2554998|
|        1|       248406442|
|        3|        22508413|
|        2|         1092048|
|        7|             869|
+---------+----------------+



Llama la atención que el tiempo de trayecto del tipo 2 "turista" sea superior en diciembre de 2020 que en 2018 puesto que en 2020 el número de trayectos efectuados por turistas ya hemos vistos que fueron muy inferior. Esto indica que, aunque fueron menos turistas, hicieron recorridos muchos más largos prescindiendo del transporte público.

# CONCLUSIONES
Tras analizar los datos hemos observado que, debido a la pandemia provocada del COVID-19, en junio de 2020 se redujeron considerablemente los viajes en el servicio de BICIMAD frente a junio de 2019. En junio de 2021, los datos mejoraron sin llegar a alcanzar las cifras de 2019. Cabe destacar que el número de usuarios de tipo "turista" en el año 2021 fue mucho menor que en el año 2019. 

Las estaciones de BICIMAD más utilizadas en junio de 2019 fueron la 175 y la 163, en junio de 2020 la 43 y 175 y en junio de 2021 la 43. Las menos utilizadas fueron en 2019 la 28 y la 207, en 2020 la 28 y la 151, y en 2021 la 28 y 257. 

Analizando las cifras del mes de diciembre de los años 2018 y 2020 nos llama la atención que, a pesar de ser un mes frío y tras un periodo de confinamiento en el que el teletrabajo y los ERTES predominaron, el número de trayectos fue mucho más elevado en 2020 que en 2018. En cuanto a los turistas, podemos observar que, a pesar de que en 2020 hubo más trayectos, estos se llevaron a cabo por menos turistas que en diciembre de 2018 por las restricciones sanitarias que impedían el turismo internacional y de algunas comunidades autónomas. 

Las estaciones más utilizadas en diciembre de 2018 fueron la 175 y 135, y las menos utilizadas 28 y 29. En diciembre de 2020 en cambio, las más utilizadas fueron 43 y 57 y las menos 257 y 249.